<a href="https://colab.research.google.com/github/zli96/CAGNET/blob/master/RewriteCagnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git


     |████████████████████████████████| 10.4 MB 5.1 MB/s 
     |████████████████████████████████| 3.7 MB 5.3 MB/s 
     |████████████████████████████████| 407 kB 5.4 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 


In [1]:
import torch
from torch.nn import Linear
import torch.nn.functional as F

class GCNFunc(torch.autograd.Function):
    @staticmethod
    def forward(ctx, inputs, weight, adj_matrix, func):
        global run
        # inputs: H
        # adj_matrix: A
        # weight: W
        # func: sigma

        # adj_matrix = adj_matrix.to_dense()
        ctx.save_for_backward(inputs, weight, adj_matrix)
        ctx.func = func

        z = torch.sparse.mm(adj_matrix, inputs)
        z = torch.mm(z, weight)

        z.requires_grad = True
        ctx.z = z

        # if activations:
        if func is F.log_softmax:
            h = func(z, dim=1)
        elif func is F.relu:
            h = func(z)
        else:
            h = z

        return h
        # else:
        #     return z

    @staticmethod
    def backward(ctx, grad_output):
        global run

        inputs, weight, adj_matrix = ctx.saved_tensors
        func = ctx.func
        z = ctx.z

        with torch.set_grad_enabled(True):
            if func is F.log_softmax:
                func_eval = func(z, dim=1)
            elif func is F.relu:
                func_eval = func(z)
            else:
                func_eval = z

            sigmap = torch.autograd.grad(outputs=func_eval, inputs=z, grad_outputs=grad_output)[0]
            grad_output = sigmap

        # First backprop 
        ag = torch.sparse.mm(adj_matrix, grad_output)

        grad_input = torch.mm(ag, weight.t())

        # Second backprop equation (reuses the A * G^l computation)
        grad_weight = torch.mm(inputs.t(), ag)

        return grad_input, grad_weight, None, None, None, None, None, None


In [3]:
torch.cuda.is_available()

True

In [4]:
def train(inputs, weight1, weight2, adj_matrix, optimizer, data):
    outputs = GCNFunc.apply(inputs, weight1, adj_matrix, F.relu)
    outputs = GCNFunc.apply(outputs, weight2, adj_matrix, F.log_softmax)

    optimizer.zero_grad()

    # Note: bool type removes warnings, unsure of perf penalty
    loss = F.nll_loss(outputs[data.train_mask.bool()], data.y[data.train_mask.bool()])
    print('loss: {:.4f}'.format(loss))
    loss.backward()

    optimizer.step()

    return outputs

In [5]:
def test(outputs, data):
    logits, accs = outputs, []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)

    if len(accs) == 1:
        accs.append(0)
        accs.append(0)

    return accs

In [9]:
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
dataset = 'Cora'
dataset = Planetoid('../data', dataset, transform=T.NormalizeFeatures())
data = dataset[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Processing...
Done!


In [12]:
from torch.nn import Parameter
import numpy as np
from scipy.sparse import coo_matrix

inputs = data
num_features = dataset.num_features
mid_layer = 16
epochs=100
num_classes=dataset.num_classes
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(0)
weight1_nonleaf = torch.rand(num_features, mid_layer, requires_grad=True)
weight1_nonleaf = weight1_nonleaf.to(device)
weight1_nonleaf.retain_grad()

weight2_nonleaf = torch.rand(mid_layer, num_classes, requires_grad=True)
weight2_nonleaf = weight2_nonleaf.to(device)
weight2_nonleaf.retain_grad()

weight1 = Parameter(weight1_nonleaf)
weight2 = Parameter(weight2_nonleaf)

optimizer = torch.optim.Adam([weight1, weight2], lr=0.01)
temp_value = np.ones((data.edge_index.shape[1],),dtype='float32')
idx_array=data.edge_index.data.numpy()
num_nodes = data.x.size()[0]
adj_matrix = torch.sparse_coo_tensor((idx_array[0,:],idx_array[1,:]), temp_value, ( num_nodes, num_nodes))
inputs.x  = inputs.x.to(device)
adj_matrix = adj_matrix.to(device)
data = data.to(device)
for epoch in range(1, epochs):
  outputs = train(inputs.x, weight1, weight2, adj_matrix, optimizer, data)
  train_acc, val_acc, test_acc = test(outputs, data)
  log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
  print(log.format(epoch, train_acc, val_acc, test_acc))

loss: 78.0500
Epoch: 001, Train: 0.1429, Val: 0.1140, Test: 0.1030
loss: 69.7698
Epoch: 002, Train: 0.1857, Val: 0.1240, Test: 0.1190
loss: 64.5411
Epoch: 003, Train: 0.1857, Val: 0.0900, Test: 0.1060
loss: 58.6881
Epoch: 004, Train: 0.1929, Val: 0.0960, Test: 0.1140
loss: 52.8371
Epoch: 005, Train: 0.2000, Val: 0.1320, Test: 0.1260
loss: 49.9903
Epoch: 006, Train: 0.1500, Val: 0.0620, Test: 0.0720
loss: 45.9134
Epoch: 007, Train: 0.1500, Val: 0.0620, Test: 0.0720
loss: 40.9685
Epoch: 008, Train: 0.1643, Val: 0.0740, Test: 0.0820
loss: 36.2557
Epoch: 009, Train: 0.2286, Val: 0.1040, Test: 0.1100
loss: 31.8722
Epoch: 010, Train: 0.3000, Val: 0.1680, Test: 0.1680
loss: 28.7828
Epoch: 011, Train: 0.3071, Val: 0.1880, Test: 0.2150
loss: 27.4489
Epoch: 012, Train: 0.1714, Val: 0.1280, Test: 0.1440
loss: 24.8376
Epoch: 013, Train: 0.1857, Val: 0.1300, Test: 0.1480
loss: 21.6600
Epoch: 014, Train: 0.2643, Val: 0.1660, Test: 0.1870
loss: 18.8792
Epoch: 015, Train: 0.2643, Val: 0.1800, Test: 0.

In [19]:
data.x.size()[0]

2708